In [13]:
# importing packages
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import subprocess
from tqdm import tqdm
import matplotlib.ticker as mticker
import cartopy.crs as ccrs
import cartopy

In [2]:
n = 600 # number of frames
lons = np.ravel([[np.linspace(0, 360, n//3)] * 3])[::-1] # three full rotations
# from equator to north pole, then to south pole and back to equator 
lats = np.hstack([np.linspace(0, 90, n//6), np.ones(n//6) * 90, np.linspace(90, -90, 2 * n//6), -np.ones(n//6) * 90, np.linspace(-90, 0, n//6)])

In [49]:
# for each frame
for i in tqdm(np.arange(n)):
    # get the central longitude and latitude
    lonc, latc = lons[i], lats[i]

    # define projection
    proj = ccrs.Orthographic(central_latitude = latc, central_longitude = lonc)

    # create figure with the chosen projection
    fig = plt.figure(figsize = (3, 3))
    ax = plt.axes(projection = proj)

    # add ocean, land and coastlines
    ax.add_feature(cartopy.feature.OCEAN)
    ax.add_feature(cartopy.feature.LAND, zorder = 10)
    ax.coastlines("110m", color = "0.2", zorder = 11)

    # add grid lines
    gl = ax.gridlines(color = "0.4", zorder = 12)
    gl.xlocator = mticker.FixedLocator(np.arange(-180, 180, 20))
    gl.ylocator = mticker.FixedLocator(np.arange(-90, 90, 20))

    # add title if at north or south poles
    title = ""
    if latc / 90 == 1:
        title = "North Pole (counter clockwise)"
    elif latc / 90 == -1:
        title = "South Pole (clockwise)"
    ax.set_title(title)

    # save the figure with a good quality
    fig.savefig(f"img/{i:05d}.png", dpi = 200)

    # close figures
    plt.close("all")

100%|█████████████████████████████████████████████████████████████████████████████████| 600/600 [02:48<00:00,  3.55it/s]


In [52]:
fps = 24 # frames per second
command = [
    'ffmpeg', # software for converting from pngs to a gif
    '-y',
    '-framerate', str(fps),  # frame rate
    '-pattern_type', 'glob',  # use glob pattern
    '-i', 'img/*.png',  # input format with glob pattern for png files
    '-filter_complex', '[0:v] split [a][b];[a] palettegen [p];[b][p] paletteuse',  # improve gif quality
    'img/output.gif'  # output as gif
]
subprocess.run(command)

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

CompletedProcess(args=['ffmpeg', '-y', '-framerate', '24', '-pattern_type', 'glob', '-i', 'img/*.png', '-filter_complex', '[0:v] split [a][b];[a] palettegen [p];[b][p] paletteuse', 'img/output.gif'], returncode=0)